In [2]:
import json
import requests
import pandas as pd
from pathlib import Path

# === Step 1: Read the token from file ===
def load_bearer_token(token_file="~/work/id_token_maint.json"):
    path = Path(token_file).expanduser()
    with open(path) as f:
        token_data = json.load(f)
    return token_data["id_token"]

BEARER_TOKEN = load_bearer_token()
HEADERS = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Accept": "application/json"
}

# === Step 2: Fetch all projects ===
def get_all_projects():
    projects = []
    url = "https://beta-3.fabric-testbed.net/projects?limit=100&offset=0"
    while url:
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            raise Exception(f"Failed to fetch projects: {resp.status_code} {resp.text}")
        data = resp.json()
        projects.extend(data.get("results", []))
        url = data.get("links", {}).get("next")
    return projects

# === Step 3: Fetch quota per project ===
def get_project_quota(project_id):
    quotas = []
    url = f"https://beta-3.fabric-testbed.net/quotas?project_uuid={project_id}&offset=0&limit=100"
    while url:
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            return [{"error": resp.text}]
        data = resp.json()
        quotas.extend(data.get("results", []))
        url = data.get("links", {}).get("next")

    return quotas

# === Step 4: Aggregate quota info ===
def fetch_all_project_quotas():
    projects = get_all_projects()
    records = []
    print(f"Total projects: {len(projects)}")
    print("Fetching quotas...")

    for proj in projects:
        project_id = proj["uuid"]
        quota = get_project_quota(project_id)
        if len(quota) > 0 and "error" not in quota[0]:
            for q in quota:
                records.append({
                    "project_id": project_id,
                    "project_name": proj.get("name"),
                    "resource_name": q["resource_type"]["name"],
                    "resource_value": q["resource_type"]["value"],
                    "quota_limit": q["quota_limit"],
                    "quota_used": q["quota_used"],
                    "resource_unit": q["resource_unit"],
                    "created_at": q["created_at"],
                    "updated_at": q["updated_at"],
                    "quota_id": q["uuid"]
                })
        else:
            records.append({
            "project_id": project_id,
            "project_name": proj.get("name"),
            "error": quota[0].get("error", "unknown") if isinstance(quota, list) and quota else "unknown"
            })

    return pd.DataFrame(records)

# === Step 5: Run and display ===
df_quotas = fetch_all_project_quotas()
display(df_quotas)


Total projects: 17
Fetching quotas...


,project_id,project_name,resource_name,resource_value,quota_limit,quota_used,resource_unit,created_at,updated_at,quota_id
0,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,Anonymization testing,p4,P4,0.0,0.0,Hours,2025-02-20 17:10:08.271524+00:00,2025-02-20 17:10:08.271524+00:00,1c7c07d3-9a5b-4efb-935c-94e2cf9d378a
1,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,Anonymization testing,core,Core,0.0,0.0,Hours,2025-02-20 17:10:08.276954+00:00,2025-02-20 17:10:08.276954+00:00,b9032e99-3315-43d8-bda2-6c4b34b72384
2,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,Anonymization testing,ram,RAM,0.0,0.0,Hours,2025-02-20 17:10:08.291712+00:00,2025-02-20 17:10:08.291712+00:00,32464dac-a5fb-47ee-aef4-6949f5006133
3,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,Anonymization testing,disk,Disk,0.0,0.0,Hours,2025-02-20 17:10:08.300349+00:00,2025-02-20 17:10:08.300349+00:00,ab872fcf-6ae0-418c-b972-21ad00b62fbe
4,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,Anonymization testing,gpu_rtx6000,GPU RTX6000,0.0,0.0,Hours,2025-02-20 17:10:08.307517+00:00,2025-02-20 17:10:08.307517+00:00,93ae8c81-bb38-42cd-b813-95468bfcf763
...,...,...,...,...,...,...,...,...,...,...
267,b83d391d-5b74-40a6-8790-872183eb0cee,The Genome Lake,smartnic_connectx_5,SmartNIC ConnectX 5,0.0,0.0,Hours,2025-02-20 17:10:09.194999+00:00,2025-02-20 17:10:09.194999+00:00,c8428f09-a80c-404c-8e74-5c6d9d558295
268,b83d391d-5b74-40a6-8790-872183eb0cee,The Genome Lake,nvme_p4510,NVME P4510,0.0,0.0,Hours,2025-02-20 17:10:09.201131+00:00,2025-02-20 17:10:09.201131+00:00,830192d4-090b-43f5-aaad-487cadf0d427
269,b83d391d-5b74-40a6-8790-872183eb0cee,The Genome Lake,storage_nas,Storage NAS,0.0,0.0,Hours,2025-02-20 17:10:09.207508+00:00,2025-02-20 17:10:09.207508+00:00,dbeb9da9-293a-48a4-92a4-2eecf2626468
270,b83d391d-5b74-40a6-8790-872183eb0cee,The Genome Lake,fpga_xilinx_u280,FPGA XILINX U280,0.0,0.0,Hours,2025-02-20 17:10:09.213751+00:00,2025-02-20 17:10:09.213751+00:00,626d18a9-9924-42f5-afe0-a94482d3b960


In [3]:
# Filter out rows with errors
df_clean = df_quotas

# Format combined string: used / limit
df_clean["used/limit"] = df_clean["quota_used"].astype(str) + " / " + df_clean["quota_limit"].astype(str)

# Build a new column header as: "resource_name [unit]"
df_clean["resource_column"] = df_clean["resource_name"] + " [used / limit, " + df_clean["resource_unit"] + "]"

# Pivot: rows = project, columns = new resource_column, values = "used / limit"
df_combined = df_clean.pivot_table(
    index=["project_name", "project_id"],
    columns="resource_column",
    values="used/limit",
    aggfunc="first"
)

# Optional: tidy and sort
df_combined.reset_index(inplace=True)
df_combined.sort_values("project_name", inplace=True)

# Display all columns/rows in notebook
import pandas as pd
from IPython.display import display
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

display(df_combined)

resource_column,project_name,project_id,"core [used / limit, Hours]","disk [used / limit, Hours]","fpga_xilinx_sn1022 [used / limit, Hours]","fpga_xilinx_u280 [used / limit, Hours]","gpu_a30 [used / limit, Hours]","gpu_a40 [used / limit, Hours]","gpu_rtx6000 [used / limit, Hours]","gpu_tesla_t4 [used / limit, Hours]","nvme_p4510 [used / limit, Hours]","p4 [used / limit, Hours]","ram [used / limit, Hours]","sharednic_connectx_6 [used / limit, Hours]","smartnic_bluefield_2_connectx_6 [used / limit, Hours]","smartnic_connectx_5 [used / limit, Hours]","smartnic_connectx_6 [used / limit, Hours]","storage_nas [used / limit, Hours]"
0,ASU-BU Astro Project,75bb6177-4166-48ec-aaac-b771f54bc7e7,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
1,Anonymization testing,0be3a5c6-fe9e-41a4-89b6-b6f7ffaa58a8,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
2,Beta Portal Test Project,689bd0e4-1bcd-472a-a500-b0fae3ca5884,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
3,Designing an autonomous self-defending network...,fb07edb4-6609-477e-a320-4b19dcf4826c,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
4,FABRIC Blue Fields,36c9fe99-098c-4896-90e7-01b23658d5b6,348.12 / 0.0,3895.8701888888886 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.7886922222222239 / 0.0,1388.676151111111 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
5,FABRIC STAR Program,27244686-8302-4c74-b808-598be32a7839,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
6,FABRIC Staff,b9847fa1-13ef-49f9-9e07-ae6ad06cda3f,192.00094333333334 / 0.0,480.0023583333334 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,768.0037733333334 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
7,FABRIC Students,ade3a7dd-9488-40f9-8fc6-967de0eea735,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
8,Ilya's test project,7a1ffe53-1945-4934-93c5-ef6df39b45ac,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
9,NetAM Testing,937bbd15-1379-4e20-b24f-fa658352eaa0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0,0.0 / 0.0
